# 11 - RAG Avanzado: Agentic RAG

## Curso de LLMs y Aplicaciones de IA

**Duración estimada:** 2.5-3 horas

---

## Índice

1. [¿Qué es Agentic RAG?](#intro)
2. [Arquitectura completa](#arquitectura)
3. [Implementación paso a paso](#implementacion)
4. [Evaluación con RAGAS](#evaluacion)
5. [Mejores prácticas de producción](#produccion)
6. [Ejercicio final](#ejercicio)

---

## Objetivos de aprendizaje

Al finalizar este notebook, serás capaz de:
- Diseñar sistemas RAG agentic completos
- Implementar evaluación de calidad con RAGAS
- Aplicar patrones de producción
- Optimizar el rendimiento del sistema

<a name="intro"></a>
## 1. ¿Qué es Agentic RAG?

**Agentic RAG** combina las capacidades de:
- **RAG**: Acceso a conocimiento externo
- **Agentes**: Razonamiento y uso de herramientas
- **Grafos**: Flujos de trabajo complejos

### Comparación

| RAG Simple | Agentic RAG |
|------------|-------------|
| Un paso de retrieval | Múltiples pasos adaptativos |
| Sin razonamiento | Planifica y razona |
| Query fija | Reformula queries |
| Sin verificación | Auto-corrige respuestas |

In [1]:
#!pip install -q langchain langchain-groq langgraph langchain-huggingface faiss-cpu

In [2]:
import os
from getpass import getpass
import warnings
warnings.filterwarnings('ignore')

if 'GROQ_API_KEY' not in os.environ:
    os.environ['GROQ_API_KEY'] = getpass("GROQ API Key: ")

print("Configurado ✓")

GROQ API Key:  ········


Configurado ✓


<a name="arquitectura"></a>
## 2. Arquitectura Completa

```
┌────────────────────────────────────────────────────────────────┐
│                      AGENTIC RAG SYSTEM                        │
├────────────────────────────────────────────────────────────────┤
│                                                                │
│  Query → [Clasificar] → [Reformular?] → [Retrieval]           │
│              ↓                              ↓                  │
│     ┌───────┴───────┐              [Verificar contexto]       │
│     │   FAQ?        │                      ↓                  │
│     │   General?    │              [Generar respuesta]        │
│     │   Técnica?    │                      ↓                  │
│     └───────────────┘              [Verificar calidad]        │
│                                            ↓                  │
│                                    [Corregir si necesario]    │
│                                            ↓                  │
│                                    [Respuesta final]          │
└────────────────────────────────────────────────────────────────┘
```

<a name="implementacion"></a>
## 3. Implementación paso a paso

In [4]:
# Setup components
from langchain_groq import ChatGroq
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_core.documents import Document
from langchain_core.messages import HumanMessage, AIMessage
from langgraph.graph import StateGraph, START, END
from langgraph.checkpoint.memory import MemorySaver
from typing import TypedDict, List, Literal

# Initialize LLM
llm = ChatGroq(model_name="llama-3.3-70b-versatile", temperature=0.3)

# Create knowledge base
knowledge_docs = [
    Document(page_content="El IBI se paga anualmente y grava bienes inmuebles urbanos y rústicos.", 
             metadata={"source": "ibi", "type": "definition"}),
    Document(page_content="El IVTM grava la titularidad de vehículos. Se paga mientras esté matriculado.",
             metadata={"source": "ivtm", "type": "definition"}),
    Document(page_content="Bonificaciones IBI: familias numerosas hasta 90%, energías renovables 50%.",
             metadata={"source": "ibi", "type": "bonus"}),
    Document(page_content="Bonificaciones IVTM: vehículos eléctricos 75%, históricos 100%.",
             metadata={"source": "ivtm", "type": "bonus"}),
    Document(page_content="Plazos de pago: IBI segundo semestre, IVTM primer trimestre.",
             metadata={"source": "general", "type": "deadlines"}),
    Document(page_content="Recursos: Puede interponer recurso en 30 días desde la notificación.",
             metadata={"source": "general", "type": "appeals"}),
]

embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
vectorstore = FAISS.from_documents(knowledge_docs, embeddings)
retriever = vectorstore.as_retriever(search_kwargs={"k": 3})

print("Componentes inicializados ✓")

Loading weights: 100%|█████████████████████████████████████████████████████████████████████| 103/103 [00:00<00:00, 972.18it/s, Materializing param=pooler.dense.weight]
BertModel LOAD REPORT from: sentence-transformers/all-MiniLM-L6-v2
Key                     | Status     |  | 
------------------------+------------+--+-
embeddings.position_ids | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.


Componentes inicializados ✓


In [5]:
# Define state
class AgenticRAGState(TypedDict):
    question: str
    question_type: str
    reformulated_question: str
    context: str
    context_sufficient: bool
    response: str
    response_quality: str
    final_response: str
    iteration: int

# Node functions
def classify_question(state: AgenticRAGState) -> AgenticRAGState:
    """Classify the type of question."""
    prompt = f"""Clasifica esta pregunta en una de las categorías: DEFINITION, BONUS, DEADLINE, APPEAL, OTHER.
    
Pregunta: {state['question']}

Responde solo con la categoría."""
    
    response = llm.invoke(prompt)
    q_type = response.content.strip().upper()
    return {**state, "question_type": q_type}

def reformulate_question(state: AgenticRAGState) -> AgenticRAGState:
    """Reformulate question for better retrieval."""
    prompt = f"""Reformula esta pregunta para mejorar la búsqueda de información.
Mantén la esencia pero hazla más específica.

Pregunta original: {state['question']}

Pregunta reformulada:"""
    
    response = llm.invoke(prompt)
    return {**state, "reformulated_question": response.content.strip()}

def retrieve_context(state: AgenticRAGState) -> AgenticRAGState:
    """Retrieve relevant documents."""
    query = state.get("reformulated_question") or state["question"]
    docs = retriever.invoke(query)
    context = "\n".join([f"- {d.page_content}" for d in docs])
    return {**state, "context": context}

def check_context(state: AgenticRAGState) -> AgenticRAGState:
    """Check if context is sufficient."""
    prompt = f"""¿El contexto contiene información relevante para responder la pregunta?
    
Pregunta: {state['question']}
Contexto: {state['context']}

Responde SI o NO."""
    
    response = llm.invoke(prompt)
    sufficient = "SI" in response.content.upper()
    return {**state, "context_sufficient": sufficient}

def generate_response(state: AgenticRAGState) -> AgenticRAGState:
    """Generate response from context."""
    prompt = f"""Responde la pregunta basándote SOLO en el contexto proporcionado.
Si no hay información suficiente, indícalo claramente.

Contexto:
{state['context']}

Pregunta: {state['question']}

Respuesta:"""
    
    response = llm.invoke(prompt)
    return {**state, "response": response.content}

def check_quality(state: AgenticRAGState) -> AgenticRAGState:
    """Check response quality."""
    prompt = f"""Evalúa la calidad de esta respuesta.
    
Pregunta: {state['question']}
Contexto usado: {state['context']}
Respuesta: {state['response']}

¿La respuesta es BUENA, MEJORABLE o MALA?"""
    
    response = llm.invoke(prompt)
    quality = "BUENA" if "BUENA" in response.content.upper() else "MEJORABLE"
    return {**state, "response_quality": quality, "iteration": state.get("iteration", 0) + 1}

def improve_response(state: AgenticRAGState) -> AgenticRAGState:
    """Improve the response."""
    prompt = f"""Mejora esta respuesta haciéndola más clara y completa.
    
Contexto: {state['context']}
Respuesta original: {state['response']}

Respuesta mejorada:"""
    
    response = llm.invoke(prompt)
    return {**state, "response": response.content}

def finalize(state: AgenticRAGState) -> AgenticRAGState:
    """Finalize the response."""
    return {**state, "final_response": state["response"]}

def no_info_response(state: AgenticRAGState) -> AgenticRAGState:
    """Response when no info found."""
    return {**state, "final_response": "Lo siento, no tengo información suficiente para responder a esa pregunta."}

# Routing functions
def route_context(state: AgenticRAGState) -> Literal["generate", "no_info"]:
    return "generate" if state["context_sufficient"] else "no_info"

def route_quality(state: AgenticRAGState) -> Literal["improve", "finalize"]:
    if state["response_quality"] != "BUENA" and state.get("iteration", 0) < 2:
        return "improve"
    return "finalize"

print("Funciones de nodo definidas ✓")

Funciones de nodo definidas ✓


In [6]:
# Build the graph
workflow = StateGraph(AgenticRAGState)

# Add nodes
workflow.add_node("classify", classify_question)
workflow.add_node("reformulate", reformulate_question)
workflow.add_node("retrieve", retrieve_context)
workflow.add_node("check_context", check_context)
workflow.add_node("generate", generate_response)
workflow.add_node("check_quality", check_quality)
workflow.add_node("improve", improve_response)
workflow.add_node("finalize", finalize)
workflow.add_node("no_info", no_info_response)

# Add edges
workflow.add_edge(START, "classify")
workflow.add_edge("classify", "reformulate")
workflow.add_edge("reformulate", "retrieve")
workflow.add_edge("retrieve", "check_context")
workflow.add_conditional_edges("check_context", route_context, 
                                {"generate": "generate", "no_info": "no_info"})
workflow.add_edge("generate", "check_quality")
workflow.add_conditional_edges("check_quality", route_quality,
                                {"improve": "improve", "finalize": "finalize"})
workflow.add_edge("improve", "check_quality")
workflow.add_edge("finalize", END)
workflow.add_edge("no_info", END)

# Compile with memory
memory = MemorySaver()
agentic_rag = workflow.compile(checkpointer=memory)

print("Agentic RAG compilado ✓")

Agentic RAG compilado ✓


In [7]:
# Test the system
def ask(question: str, session_id: str = "default"):
    config = {"configurable": {"thread_id": session_id}}
    result = agentic_rag.invoke({
        "question": question,
        "question_type": "",
        "reformulated_question": "",
        "context": "",
        "context_sufficient": False,
        "response": "",
        "response_quality": "",
        "final_response": "",
        "iteration": 0
    }, config=config)
    
    print(f"\n📝 Pregunta: {question}")
    print(f"🏷️ Tipo: {result['question_type']}")
    print(f"🔄 Reformulada: {result['reformulated_question']}")
    print(f"✅ Contexto suficiente: {result['context_sufficient']}")
    print(f"⭐ Calidad: {result['response_quality']}")
    print(f"🔢 Iteraciones: {result['iteration']}")
    print(f"\n💬 Respuesta final:\n{result['final_response']}")
    return result

# Test
ask("¿Qué bonificaciones hay para vehículos eléctricos?")


📝 Pregunta: ¿Qué bonificaciones hay para vehículos eléctricos?
🏷️ Tipo: BONUS
🔄 Reformulada: Pregunta reformulada: ¿Cuáles son las bonificaciones y subsidios gubernamentales actuales para la compra y uso de vehículos eléctricos en mi país o región?
✅ Contexto suficiente: True
⭐ Calidad: BUENA
🔢 Iteraciones: 1

💬 Respuesta final:
Hay una bonificación del 75% para vehículos eléctricos en el IVTM.


{'question': '¿Qué bonificaciones hay para vehículos eléctricos?',
 'question_type': 'BONUS',
 'reformulated_question': 'Pregunta reformulada: ¿Cuáles son las bonificaciones y subsidios gubernamentales actuales para la compra y uso de vehículos eléctricos en mi país o región?',
 'context': '- Bonificaciones IVTM: vehículos eléctricos 75%, históricos 100%.\n- El IVTM grava la titularidad de vehículos. Se paga mientras esté matriculado.\n- El IBI se paga anualmente y grava bienes inmuebles urbanos y rústicos.',
 'context_sufficient': True,
 'response': 'Hay una bonificación del 75% para vehículos eléctricos en el IVTM.',
 'response_quality': 'BUENA',
 'final_response': 'Hay una bonificación del 75% para vehículos eléctricos en el IVTM.',
 'iteration': 1}

In [8]:
# Test more questions
questions = [
    "¿Cuándo se paga el IBI?",
    "¿Puedo recurrir una liquidación?",
    "¿Qué descuentos hay para familias numerosas?"
]

for q in questions:
    ask(q)
    print("\n" + "="*60)


📝 Pregunta: ¿Cuándo se paga el IBI?
🏷️ Tipo: DEADLINE
🔄 Reformulada: Pregunta reformulada: ¿Cuál es el plazo y la fecha límite para el pago del Impuesto sobre Bienes Inmuebles (IBI) en España?
✅ Contexto suficiente: True
⭐ Calidad: BUENA
🔢 Iteraciones: 1

💬 Respuesta final:
Según el contexto proporcionado, el IBI se paga anualmente, específicamente en el segundo semestre.


📝 Pregunta: ¿Puedo recurrir una liquidación?
🏷️ Tipo: APPEAL
🔄 Reformulada: Pregunta reformulada: ¿Cuáles son los pasos y requisitos legales para recurrir una liquidación de una empresa o una sentencia de liquidación en un tribunal?
✅ Contexto suficiente: True
⭐ Calidad: BUENA
🔢 Iteraciones: 1

💬 Respuesta final:
Sí, puedes recurrir una liquidación. Tienes 30 días desde la notificación para interponer el recurso.


📝 Pregunta: ¿Qué descuentos hay para familias numerosas?
🏷️ Tipo: OTHER
🔄 Reformulada: Pregunta reformulada: ¿Cuáles son los beneficios y descuentos fiscales o de servicios públicos disponibles para fami

<a name="evaluacion"></a>
## 4. Evaluación con métricas

Evaluamos la calidad del sistema RAG con métricas clave.

In [9]:
# Simple evaluation without RAGAS (to avoid dependency issues)
def evaluate_response(question, response, context, ground_truth=None):
    """Simple evaluation of RAG response quality."""
    
    # Check if response is grounded in context (faithfulness)
    faithfulness_prompt = f"""¿La respuesta está basada en el contexto proporcionado?
    
Contexto: {context}
Respuesta: {response}

Puntúa de 0 a 1 (donde 1 es completamente basada en el contexto)."""
    
    faith_score = llm.invoke(faithfulness_prompt)
    
    # Check relevance
    relevance_prompt = f"""¿La respuesta es relevante para la pregunta?
    
Pregunta: {question}
Respuesta: {response}

Puntúa de 0 a 1."""
    
    rel_score = llm.invoke(relevance_prompt)
    
    return {
        "faithfulness_eval": faith_score.content,
        "relevance_eval": rel_score.content
    }

# Evaluate one response
test_result = ask("¿Qué es el IBI?")
eval_result = evaluate_response(
    test_result["question"],
    test_result["final_response"],
    test_result["context"]
)

print("\n📊 Evaluación:")
print(f"Fidelidad: {eval_result['faithfulness_eval']}")
print(f"Relevancia: {eval_result['relevance_eval']}")


📝 Pregunta: ¿Qué es el IBI?
🏷️ Tipo: DEFINITION
🔄 Reformulada: Pregunta reformulada: ¿Qué es el Impuesto sobre Bienes Inmuebles (IBI) y cómo se calcula en España?
✅ Contexto suficiente: True
⭐ Calidad: BUENA
🔢 Iteraciones: 1

💬 Respuesta final:
El IBI (Impuesto de Bienes Inmuebles) es un impuesto que se paga anualmente y grava bienes inmuebles urbanos y rústicos.

📊 Evaluación:
Fidelidad: La puntuación sería 1, ya que la respuesta está completamente basada en el contexto proporcionado. La oración "El IBI se paga anualmente y grava bienes inmuebles urbanos y rústicos" se encuentra directamente en el contexto, lo que indica que la respuesta es una reproducción exacta de la información proporcionada.
Relevancia: La respuesta es completamente relevante para la pregunta, ya que define claramente qué es el IBI y a qué se aplica. Por lo tanto, le daría una puntuación de 1.


<a name="produccion"></a>
## 5. Mejores prácticas de producción

### Checklist para producción

- [ ] **Logging**: Registrar todas las consultas y respuestas
- [ ] **Métricas**: Monitorear latencia, calidad, uso
- [ ] **Rate limiting**: Controlar costos de API
- [ ] **Fallbacks**: Respuestas por defecto si falla
- [ ] **Caching**: Cachear respuestas frecuentes
- [ ] **Testing**: Tests automatizados de regresión

In [10]:
# Example: Add logging wrapper
import logging
from datetime import datetime

logging.basicConfig(level=logging.INFO)
logger = logging.getLogger("AgenticRAG")

def ask_with_logging(question: str, session_id: str = "default"):
    """Wrapper with logging for production."""
    start_time = datetime.now()
    logger.info(f"Query received: {question}")
    
    try:
        result = ask(question, session_id)
        
        elapsed = (datetime.now() - start_time).total_seconds()
        logger.info(f"Response generated in {elapsed:.2f}s")
        logger.info(f"Quality: {result['response_quality']}")
        
        return result
    except Exception as e:
        logger.error(f"Error processing query: {e}")
        return {"final_response": "Lo siento, ocurrió un error. Intenta de nuevo."}

# Test
ask_with_logging("¿Cuáles son los plazos de pago?")

INFO:AgenticRAG:Query received: ¿Cuáles son los plazos de pago?
INFO:httpx:HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 429 Too Many Requests"
INFO:groq._base_client:Retrying request to /openai/v1/chat/completions in 2.000000 seconds
INFO:httpx:HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 429 Too Many Requests"
INFO:groq._base_client:Retrying request to /openai/v1/chat/completions in 2.000000 seconds
INFO:httpx:HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"
INFO:AgenticRAG:Response generated in 5.04s
INFO:Age


📝 Pregunta: ¿Cuáles son los plazos de pago?
🏷️ Tipo: DEADLINE
🔄 Reformulada: Pregunta reformulada: ¿Cuáles son los plazos y condiciones específicas de pago que se aplican a este servicio o producto?
✅ Contexto suficiente: True
⭐ Calidad: BUENA
🔢 Iteraciones: 1

💬 Respuesta final:
Los plazos de pago son:
- IBI: segundo semestre
- IVTM: primer trimestre


{'question': '¿Cuáles son los plazos de pago?',
 'question_type': 'DEADLINE',
 'reformulated_question': 'Pregunta reformulada: ¿Cuáles son los plazos y condiciones específicas de pago que se aplican a este servicio o producto?',
 'context': '- El IVTM grava la titularidad de vehículos. Se paga mientras esté matriculado.\n- Plazos de pago: IBI segundo semestre, IVTM primer trimestre.\n- Recursos: Puede interponer recurso en 30 días desde la notificación.',
 'context_sufficient': True,
 'response': 'Los plazos de pago son:\n- IBI: segundo semestre\n- IVTM: primer trimestre',
 'response_quality': 'BUENA',
 'final_response': 'Los plazos de pago son:\n- IBI: segundo semestre\n- IVTM: primer trimestre',
 'iteration': 1}

<a name="ejercicio"></a>
## 6. Ejercicio Final

### Mejora el sistema Agentic RAG

Añade las siguientes mejoras:
1. Citación de fuentes en la respuesta
2. Detección de idioma
3. Historial de conversación

In [11]:
# Exercise: Implement improvements
# 1. Modify generate_response to include source citations
# 2. Add a language detection node
# 3. Implement conversation history

## Resumen

En este notebook hemos construido un sistema **Agentic RAG** completo:

1. **Clasificación** de preguntas
2. **Reformulación** para mejor retrieval
3. **Verificación** de contexto suficiente
4. **Generación** de respuestas
5. **Auto-corrección** iterativa
6. **Evaluación** de calidad

Este patrón es la base para asistentes de IA en producción.

En el siguiente y último notebook veremos **Modelos de Gran Contexto y Multimodales**.

---

## Referencias

- [LangGraph Agentic RAG](https://langchain-ai.github.io/langgraph/tutorials/rag/langgraph_agentic_rag/)
- [RAGAS Evaluation](https://docs.ragas.io/)

In [12]:
import session_info
session_info.show(html=False)

-----
ipykernel                   7.2.0
langchain_community         0.4.1
langchain_core              1.2.9
langchain_groq              1.1.2
langchain_huggingface       NA
langgraph                   NA
session_info                v1.0.1
-----
IPython             9.10.0
jupyter_client      8.8.0
jupyter_core        5.9.1
-----
Python 3.13.1 (tags/v3.13.1:0671451, Dec  3 2024, 19:06:28) [MSC v.1942 64 bit (AMD64)]
Windows-11-10.0.26200-SP0
-----
Session information updated at 2026-02-10 07:09
